In [2]:
pip install opencv-python opencv-python-headless

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
import time  # Import time for tracking duration

In [4]:
# Step 2: Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video stream. Please check if your camera is connected and not in use by another application.")
    exit()

print("Camera opened successfully!")

Camera opened successfully!


In [5]:
# Step 3: Load YOLO Model
config_path = "C:/Users/pooja/Downloads/yolov3.cfg"
weights_path = "C:/Users/pooja/Downloads/yolov3.weights"
net = cv2.dnn.readNet(weights_path, config_path)

# Get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

print(f"Output Layers: {output_layers}")  # Print output layer names for debugging

Output Layers: ['yolo_82', 'yolo_94', 'yolo_106']


In [ ]:
# Step 3: Capture Frame and Process Image in Real-Time
while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame.")
        break

    height, width, _ = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    # Step 4: Process YOLO Output
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.3:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.3, nms_threshold=0.4)

    # Step 5: Draw Bounding Boxes and Estimate Dimensions
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Estimating dimensions
            ref_width = 10  # cm
            pixel_per_metric = w / ref_width
            real_width = w / pixel_per_metric
            real_height = h / pixel_per_metric
            real_length = real_width * 1.5  # Assumed length-to-width ratio

            cv2.putText(frame, f"W: {real_width:.2f} cm, H: {real_height:.2f} cm", 
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Suggest box size
            box_sizes = [
                {"length": 15, "width": 10, "height": 5},
                {"length": 20, "width": 15, "height": 10},
                {"length": 30, "width": 20, "height": 15},
            ]

            def suggest_box_size(length, width, height):
                for box in box_sizes:
                    if length <= box["length"] and width <= box["width"] and height <= box["height"]:
                        return box
                return None

            suggested_box = suggest_box_size(real_length, real_width, real_height)
            if suggested_box:
                print(f"Suggested Box Size: {suggested_box}")

    # Step 6: Show the frame with bounding boxes
    cv2.imshow("Frame", frame)

    # Exit when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 15, 'width': 10, 'height': 5}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 15, 'width': 10, 'height': 5}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 30, 'width': 20, 'height': 15}
Suggested Box Size: {'length': 20, 'width': 15, 'height': 10}
Suggested Box Size: {'length': 30, 'width': 20, 'height': 15}
Suggested 